# Simple contrader that can trade live and from past data

What to do: 
- Make a 'reader' that reads the csv and gives it one at a time like the api
- Make a con trader

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tpqoa
from scipy.optimize import brute
plt.style.use("seaborn")

In [3]:
api = tpqoa.tpqoa("../oanda_practice.cfg")
df_ask = api.get_history(instrument = "EUR_USD", start = "2021-01-01", end = "2021-01-08",
                granularity = "S5", price = "A")
df_bid = api.get_history(instrument = "EUR_USD", start = "2021-01-01", end = "2021-01-08",
                granularity = "S5", price = "B")

In [4]:
df_ask

,o,h,l,c,volume,complete
time,,,,,,
2021-01-03 22:00:00,1.22415,1.22415,1.22415,1.22415,1,True
2021-01-03 22:00:05,1.22415,1.22415,1.22414,1.22414,3,True
2021-01-03 22:00:30,1.22417,1.22417,1.22417,1.22417,1,True
2021-01-03 22:00:40,1.22430,1.22430,1.22423,1.22423,2,True
2021-01-03 22:01:20,1.22415,1.22415,1.22415,1.22415,2,True
...,...,...,...,...,...,...
2021-01-07 23:58:45,1.22702,1.22702,1.22702,1.22702,1,True
2021-01-07 23:58:55,1.22703,1.22703,1.22701,1.22701,2,True
2021-01-07 23:59:10,1.22703,1.22703,1.22700,1.22700,3,True


In [5]:
df_bid

,o,h,l,c,volume,complete
time,,,,,,
2021-01-03 22:00:00,1.22384,1.22384,1.22384,1.22384,1,True
2021-01-03 22:00:05,1.22372,1.22382,1.22372,1.22372,3,True
2021-01-03 22:00:30,1.22388,1.22388,1.22388,1.22388,1,True
2021-01-03 22:00:40,1.22387,1.22388,1.22387,1.22388,2,True
2021-01-03 22:01:20,1.22377,1.22387,1.22377,1.22387,2,True
...,...,...,...,...,...,...
2021-01-07 23:58:45,1.22685,1.22685,1.22685,1.22685,1,True
2021-01-07 23:58:55,1.22687,1.22687,1.22687,1.22687,2,True
2021-01-07 23:59:10,1.22688,1.22688,1.22685,1.22685,3,True


# Con trader

In [6]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.position = 0
        self.instrument = instrument
        self.window = window
        self.bar_length = bar_length
        self.units = units
        self.tick_data = pd.DataFrame()
        self.min_length = self.window + 1
        self.profits = [] # store p&l for all trades
    
    def resample_data(self):
        self.data = self.tick_data.resample(self.bar_length, label = "right").last().ffill().iloc[:-1]
    
    def prepare_data(self):
        self.data["returns"] = np.log(self.data.mid / self.data.mid.shift(1))
        self.data["position"] = -np.sign(self.data.returns.rolling(self.window).mean())
    
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
        # store and resample tick data
        df = pd.DataFrame({"bid":bid, "ask":ask, "mid":(ask + bid)/2}, 
                          index = [pd.to_datetime(time)])
        self.tick_data = self.tick_data.append(df)
        self.resample_data()
        
        # prepare data & define strategy
        self.prepare_data()
        
        # executing trades
        if len(self.data) > self.min_length - 1:
            self.min_length += 1
            if self.data["position"].iloc[-1] == 1:
                if self.position == 0:
                    order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING LONG")
                elif self.position == -1:
                    order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                    self.report_trade(order, "GOING LONG")
                self.position = 1
            elif self.data["position"].iloc[-1] == -1: 
                if self.position == 0:
                    order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                elif self.position == 1:
                    order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                    self.report_trade(order, "GOING SHORT")
                self.position = -1
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")